In [174]:
import jieba
import pandas as pd
import jieba.analyse
import jieba.posseg as posg
import random
import codecs
import re
import json
import sys

In [88]:
data_with_tag = pd.read_csv('data/data_with_tag.csv')
data_with_tag

,Unnamed: 0,data,tag
0,0,随着年纪额增长，我眼角的皱纹长起来了，加上自己不好的作息习惯，更加加速了衰老，最近想进行面部...,手术;面部除皱;手术方式
1,1,鼻翼长得非常宽大，想通过手术的方法缩小，请问医生鼻翼可以通过手术缩小吗？,手术
2,2,我同事想去做自体脂肪丰胸手术，担心自己身上的钱不够，请问自体脂肪丰胸手术的价格多少钱？,自体脂肪丰胸;手术
3,3,做假体隆胸取出哪个医院比较好？,假体隆胸;医院
4,4,最近一直在想给自己隆鼻的问题，朋友们推荐我做注射隆鼻，那么我想问一下注射隆鼻的副作用是什么？,注射隆鼻;隆鼻;副作用
...,...,...,...
745252,745252,想要尝试一下隆胸，但是又怕别的材料不安全，想要尝试一下自体脂肪隆胸，自体脂肪隆胸需要多少钱呢？,自体脂肪隆胸;隆胸
745253,745253,我的下巴看上去一点都不好看，想去做丰下巴手术，做这种手术需要多少钱呢？,手术;丰下巴
745254,745254,我因为工作性质的关系，没有太多的时间运动，现在小腿部位特别粗，一直想去做一个botox瘦小腿...,手术;瘦小腿
745255,745255,颧骨颧弓整形的费用是多少？,颧骨颧弓整形;费用


In [89]:
data_list = data_with_tag['data'].to_list()
tag_list = data_with_tag['tag'].to_list()

In [90]:
keyword_lists = []
for i in range(len(data_list)):
    keyword_list = []
    kw=jieba.analyse.extract_tags(data_list[i],topK=10,withWeight=False)
    for item in kw:
        keyword_list.append(item)
    keyword_lists.append(keyword_list)

In [91]:
locations = []
with open('./data/locations.txt','r',encoding='utf-8-sig') as f:
    for line in f.readlines():
        line = line.strip()
        locations.append(line)

In [92]:
len(locations)

362

In [93]:
#获取语料实体对列表
word_lists = []
for i in range(len(keyword_lists)):
    word_list = []
    for word in keyword_lists[i]:
        if word in locations:
            word_list.append(word)
    if word_list != []:
        word_lists.append(';'.join(word_list))
    else:
        word_lists.append('')

In [94]:
city_index = []
city_list = []
for i in range(len(word_lists)):
    if word_lists[i] != '' and len(word_lists[i].split(';'))==1:
        city_index.append(i)
        city_list.append(word_lists[i])

In [95]:
data = {'data':data_list,'tag':tag_list,'city':word_lists}
new_data = pd.DataFrame(data)

In [96]:
new_data

,data,tag,city
0,随着年纪额增长，我眼角的皱纹长起来了，加上自己不好的作息习惯，更加加速了衰老，最近想进行面部...,手术;面部除皱;手术方式,
1,鼻翼长得非常宽大，想通过手术的方法缩小，请问医生鼻翼可以通过手术缩小吗？,手术,
2,我同事想去做自体脂肪丰胸手术，担心自己身上的钱不够，请问自体脂肪丰胸手术的价格多少钱？,自体脂肪丰胸;手术,
3,做假体隆胸取出哪个医院比较好？,假体隆胸;医院,
4,最近一直在想给自己隆鼻的问题，朋友们推荐我做注射隆鼻，那么我想问一下注射隆鼻的副作用是什么？,注射隆鼻;隆鼻;副作用,
...,...,...,...
745252,想要尝试一下隆胸，但是又怕别的材料不安全，想要尝试一下自体脂肪隆胸，自体脂肪隆胸需要多少钱呢？,自体脂肪隆胸;隆胸,
745253,我的下巴看上去一点都不好看，想去做丰下巴手术，做这种手术需要多少钱呢？,手术;丰下巴,
745254,我因为工作性质的关系，没有太多的时间运动，现在小腿部位特别粗，一直想去做一个botox瘦小腿...,手术;瘦小腿,
745255,颧骨颧弓整形的费用是多少？,颧骨颧弓整形;费用,


In [117]:
#数据增强一
add_data = []
add_tag = []
add_city = []
for i in range(len(city_index)):
    for j in range(5):
        idx = random.randint(0,361)
        if city_list[i] != locations[idx]:
            add_data.append(data_list[city_index[i]].replace(city_list[i],locations[idx]))
            add_tag.append(tag_list[city_index[i]])
            add_city.append(locations[idx])

In [122]:
add_data_df = {'data':add_data,'tag':add_tag,'city':add_city}
add_data_df = pd.DataFrame(add_data_df)

In [125]:
add_data_df[:10]

,data,tag,city
0,我几天前去咨询了做双眼皮，还是比较感兴趣，但是我很害怕手术的风险，也害怕手术完了以后，眼睛更...,做双眼皮;手术,阿里
1,我几天前去咨询了做双眼皮，还是比较感兴趣，但是我很害怕手术的风险，也害怕手术完了以后，眼睛更...,做双眼皮;手术,巴彦淖尔
2,我几天前去咨询了做双眼皮，还是比较感兴趣，但是我很害怕手术的风险，也害怕手术完了以后，眼睛更...,做双眼皮;手术,榆林
3,我几天前去咨询了做双眼皮，还是比较感兴趣，但是我很害怕手术的风险，也害怕手术完了以后，眼睛更...,做双眼皮;手术,定西
4,我几天前去咨询了做双眼皮，还是比较感兴趣，但是我很害怕手术的风险，也害怕手术完了以后，眼睛更...,做双眼皮;手术,眉山
5,您好大夫：我是莆田的，在莆田做超声法瘦脸需要多少钱啊？请教您。,超声法瘦脸,莆田
6,您好大夫：我是曲靖的，在曲靖做超声法瘦脸需要多少钱啊？请教您。,超声法瘦脸,曲靖
7,您好大夫：我是柳州的，在柳州做超声法瘦脸需要多少钱啊？请教您。,超声法瘦脸,柳州
8,您好大夫：我是怀化的，在怀化做超声法瘦脸需要多少钱啊？请教您。,超声法瘦脸,怀化
9,您好大夫：我是湘潭的，在湘潭做超声法瘦脸需要多少钱啊？请教您。,超声法瘦脸,湘潭


In [100]:
all_index = [i for i in range(len(data_list))]
rest_index = list(set(all_index).difference(set(city_index)))

In [101]:
first = ['坐标xx','最近住在xx','最近一直在xx','我是xx人','我是xx地区的人','我是一个土生土长的xx人','我在xx工作','我在xx上学']
end = ['xx哪里效果好','xx哪家医院做口碑较好？','xx这边有推荐的医院吗？','xx这边有没有好一点的整形医院？','请问在xx哪里效果好','请问xx哪家医院做口碑较好？','推荐一下xx这边的医院']

In [102]:
for i in range(100000):
    j = random.randint(0,7)
    k = random.randint(0,361)
    data_list[rest_index[i]] = first[j].replace('xx',locations[k]) + ',' + data_list[rest_index[i]]
    word_lists[rest_index[i]] = locations[k]

In [103]:
data_list[rest_index[1]]

'我在上饶工作,鼻翼长得非常宽大，想通过手术的方法缩小，请问医生鼻翼可以通过手术缩小吗？'

In [104]:
for i in range(100000,200000):
    j = random.randint(0,6)
    k = random.randint(0,361)
    data_list[rest_index[i]] = data_list[rest_index[i]] + end[j].replace('xx',locations[k])
    word_lists[rest_index[i]] = locations[k]

In [105]:
data_list[120000],len(data_list),len(set(data_list))

('女友脸上出现了很多的斑点，比较难看，想要去做激光祛斑，请问一下激光祛斑痛吗？铁岭哪家医院做口碑较好？', 745257, 745257)

In [106]:
tag_list[120000]

'激光祛斑'

In [107]:
word_lists[120000]

'铁岭'

In [108]:
enhanced_data = {'data':data_list,'tag':tag_list,'city':word_lists}
enhanced_data_df = pd.DataFrame(enhanced_data)

In [112]:
len(enhanced_data_df)

745257

In [118]:
data_list += add_data
tag_list += add_tag
word_lists += add_city

In [126]:
len(data_list),len(set(data_list))

(952875, 951618)

In [128]:
new_data_index = [i for i in range(len(data_list))]

In [129]:
random.shuffle(new_data_index)

In [130]:
new_data_list = []
new_tag_list = []
new_city_list = []
for i in range(len(data_list)):
    new_data_list.append(data_list[new_data_index[i]])
    new_tag_list.append(tag_list[new_data_index[i]])
    new_city_list.append(word_lists[new_data_index[i]])

In [132]:
new_data_dict = {'data':new_data_list,'tag':new_tag_list,'city':new_city_list}
new_data_df = pd.DataFrame(new_data_dict)

,data,tag,city
20,我天生鼻骨就属于特别奇怪的，现在好不容易成年了，想在岳阳做一个鼻骨矫正手术，想知道去哪里做好？,鼻骨矫正,岳阳
21,我的乳房特别大，让我感到很尴尬，想去做巨乳缩小手术，肇庆做这种手术价格贵不贵呢？,巨乳缩小手术;贵不贵;手术,肇庆
22,生完孩子以后胸小了很多，而且有点干瘪的感觉，想要隆胸让自己的胸部变得更饱满一些，请问昆明隆胸...,隆胸,昆明
23,我脸上把大概从我九岁的时候就开始出现了雀斑，脸颊的两侧和鼻子上面还蛮多的都是小点点，我妈妈脸...,雀斑,
24,我想要去做绣眉手术，因为我的眉毛太少了，不好看，画眉又很麻烦，请问绣眉后多久可以沾水？,绣眉手术;绣眉,
25,医生好，我咨询一下瘦脸针。我女朋友是崇左的，打算做一针瘦脸。能否介绍家最好的一针瘦脸的整形医...,瘦脸;瘦脸针,崇左
26,做单颗牙种植的缺点都有哪些呢？,单颗牙种植;缺点,
27,脸上有明显的法令纹，想了解一下去法令纹也能用激光治疗吗？,法令纹;去法令纹,
28,做抽脂手术后有效果吗？,抽脂;手术,
29,医生你好，我的脸比较大，请问一下注射瘦脸针会明显达到瘦脸效果吗？,注射瘦脸针;瘦脸;医生,


In [136]:
new_data_df.to_csv('data/data_with_tag_and_city.csv')

In [175]:
"""
按照标点符号切割的预处理数据
"""
len_treshold = 108 #  每条数据的最大长度, 留下两个位置给[CLS]和[SEP]
data_dir = 'data/'
all_data_dir = 'data/data_with_tag_and_city.csv'
all_data_df = pd.read_csv(all_data_dir,encoding='utf-8')


In [176]:
all_data_df['data']

0                我有严重的眼袋，特别影响个人形象气质，想去做去眼袋手术，济南做这种手术需要多少钱呢？
1         我老公一直抱怨我的鼻子不好看，我的鼻子天生的鼻尖有点翘，鼻梁并不算太高，医生说可以做隆鼻手术...
2                               刚割的双眼皮抽脂少了还能修复不玉溪哪家医院做口碑较好？
3         身体特别的肥胖，所以想减肥，但是但是减肥的方法实在有太多了，不知道如何选择。不知道如何减肥效...
4         我的胸一直都是A，感觉像没发育的一样，虽然以前不是很注意，但是发现男生都还是喜欢胸大的，女士...
                                ...                        
952870                    我在柳州上学,鼻孔缩小术的恢复时间？我准备进行手术了，怕耽误时间。
952871    因为我的体毛比较的长，加上夏天一穿上短袖什么的露出来的手啊脚啊的。都会看到我那明显的体息，我...
952872             我腰腹部位脂肪堆积的问题比较严重，去做了腰腹吸脂手术，腰腹吸脂手术后事项有哪些？
952873                                        半永久纹眉手术，要洗几次？
952874    武汉治软疣好的医院？请问一下武汉治软疣好的医院.我的手长了个软疣，本来想去医院的，但想想不太...
Name: data, Length: 952875, dtype: object

In [177]:
# 找出所有的非中文、非英文和非数字符号
additional_chars = set()
for t in list(all_data_df['data']):
    additional_chars.update(re.findall(u'[^\u4e00-\u9fa5a-zA-Z0-9\*]', str(t)))

# 一些需要保留的符号
extra_chars = set("!#$%&\()*+,-./:;<=>?@[\\]^_`{|}~！#￥%&？《》{}“”，：‘’。（）·、；【】")
print(extra_chars)
additional_chars = additional_chars.difference(extra_chars)
# additional_chars

{'*', '<', '%', '？', '(', ':', '》', '‘', '·', '^', '/', '#', '~', '+', '，', '$', '；', '?', '}', '>', '&', '：', '-', ')', '’', '。', '）', '【', '|', '】', ',', '、', '《', ']', '@', '！', '￥', '\\', '.', '“', '”', '（', '`', '_', '{', '!', ';', '[', '='}


In [178]:
def stop_words(x):
    try:
        x = x.strip()
    except:
        return ''
    x = re.sub('{IMG:.?.?.?}', '', x)
    x = re.sub('<!--IMG_\d+-->', '', x)
    x = re.sub('(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]', '', x)  # 过滤网址
    x = re.sub('<a[^>]*>', '', x).replace("</a>", "")  # 过滤a标签
    x = re.sub('<P[^>]*>', '', x).replace("</P>", "")  # 过滤P标签
    x = re.sub('<strong[^>]*>', ',', x).replace("</strong>", "")  # 过滤strong标签
    x = re.sub('<br>', ',', x)  # 过滤br标签
    x = re.sub('www.[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]', '', x).replace("()", "")  # 过滤www开头的网址
    x = re.sub('\s', '', x)   # 过滤不可见字符
    x = re.sub('Ⅴ', 'V', x)

    for wbad in additional_chars:
        x = x.replace(wbad, '')
    return x

In [179]:
# 清除噪声
all_data_df['data'] = all_data_df['data'].apply(stop_words)
all_data_df = all_data_df.fillna('')
all_data_df

,Unnamed: 0,data,tag,city
0,0,我有严重的眼袋，特别影响个人形象气质，想去做去眼袋手术，济南做这种手术需要多少钱呢？,去眼袋手术;手术,济南
1,1,我老公一直抱怨我的鼻子不好看，我的鼻子天生的鼻尖有点翘，鼻梁并不算太高，医生说可以做隆鼻手术...,隆鼻;手术,
2,2,刚割的双眼皮抽脂少了还能修复不玉溪哪家医院做口碑较好？,抽脂;双眼皮,玉溪
3,3,身体特别的肥胖，所以想减肥，但是但是减肥的方法实在有太多了，不知道如何选择。不知道如何减肥效...,减肥,
4,4,我的胸一直都是A，感觉像没发育的一样，虽然以前不是很注意，但是发现男生都还是喜欢胸大的，女士...,胶原蛋白丰胸;并发症,
...,...,...,...,...
952870,952870,"我在柳州上学,鼻孔缩小术的恢复时间？我准备进行手术了，怕耽误时间。",鼻孔缩小术;恢复时间;手术,柳州
952871,952871,因为我的体毛比较的长，加上夏天一穿上短袖什么的露出来的手啊脚啊的。都会看到我那明显的体息，我...,脱毛,
952872,952872,我腰腹部位脂肪堆积的问题比较严重，去做了腰腹吸脂手术，腰腹吸脂手术后事项有哪些？,腰腹吸脂手术,
952873,952873,半永久纹眉手术，要洗几次？,半永久纹眉;手术,


In [185]:
lengths = []
for i in range(len(all_data_df['data'])):
    lengths.append(len(all_data_df['data'][i]))

new_text_list = []
new_project_list = []
new_cities_list = []
for i in range(len(lengths)):
    if lengths[i] <= len_treshold:
        new_text_list.append(all_data_df['data'][i])
        new_project_list.append(all_data_df['tag'][i])
        new_cities_list.append(all_data_df['city'][i])

new_lengths = []
for i in range(len(new_text_list)):
    new_lengths.append(len(new_text_list[i]))
max(new_lengths)

108

In [186]:
len(new_text_list),len(new_project_list),len(new_cities_list)

(922159, 922159, 922159)

In [188]:
data_dict = {'text':new_text_list,'project':new_project_list,'city':new_cities_list}
data_df = pd.DataFrame(data_df)

In [192]:
# 构造训练集、验证集与测试集
with codecs.open(data_dir + 'all_data.txt', 'w', encoding='utf-8') as up:
    for row in data_df.iloc[:].itertuples():
        # print(row.unknownEntities)
        text_lbl = str(row.text)
        entitys = str(row.project).split(';')
        cities = str(row.city).split(';')
        for entity in entitys:
            text_lbl = text_lbl.replace(entity, 'Ё' + (len(entity) - 2) * 'Ж' + '✈')

        for city in cities:
            if city != '':
                text_lbl = text_lbl.replace(city, '♝' + (len(city)-2) * '✺' + '♞' )

        for c1, c2 in zip(str(row.text), text_lbl):
            if c2 == 'Ё':
                up.write('{0} {1}\n'.format(c1, 'B-ORG'))
            elif c2 == 'Ж':
                up.write('{0} {1}\n'.format(c1, 'I-ORG'))
            elif c2 == '✈':
                up.write('{0} {1}\n'.format(c1, 'E-ORG'))
            elif c2 == '♝':
                up.write('{0} {1}\n'.format(c1, 'B-LOC'))
            elif c2 == '✺':
                up.write('{0} {1}\n'.format(c1, 'I-LOC'))
            elif c2 == '♞':
                up.write('{0} {1}\n'.format(c1, 'E-LOC'))
            else:
                up.write('{0} {1}\n'.format(c1, 'O'))
        up.write('\n')